<a href="https://colab.research.google.com/github/tanisaur/ML_zoomcamp/blob/main/ML_HW_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Homework 06 Trees

In [1]:
!wget https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv

--2024-11-04 00:03:05--  https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv [following]
--2024-11-04 00:03:05--  https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391501 (382K) [text/plain]
Saving to: ‘jamb_exam_results.csv’

jamb_exam_results.c 100%[===================>] 382.33K  --.-KB/s    in 0.06s   

2024-11-04 00:03:06 (6.04 MB/s) - ‘jamb_exam_result

In [3]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.tree import export_text
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import xgboost as xgb

##Prepare the dataset

In [5]:
df = pd.read_csv('jamb_exam_results.csv')
df.head()

,JAMB_Score,Study_Hours_Per_Week,Attendance_Rate,Teacher_Quality,Distance_To_School,School_Type,School_Location,Extra_Tutorials,Access_To_Learning_Materials,Parent_Involvement,IT_Knowledge,Student_ID,Age,Gender,Socioeconomic_Status,Parent_Education_Level,Assignments_Completed
0,192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,1,17,Male,Low,Tertiary,2
1,207,14,88,4,2.7,Public,Rural,No,Yes,High,High,2,15,Male,High,NaN,1
2,182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,3,20,Female,High,Tertiary,2
3,210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,4,22,Female,Medium,Tertiary,1
4,199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,5,22,Female,Medium,Tertiary,1


In [7]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.head()

,jamb_score,study_hours_per_week,attendance_rate,teacher_quality,distance_to_school,school_type,school_location,extra_tutorials,access_to_learning_materials,parent_involvement,it_knowledge,student_id,age,gender,socioeconomic_status,parent_education_level,assignments_completed
0,192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,1,17,Male,Low,Tertiary,2
1,207,14,88,4,2.7,Public,Rural,No,Yes,High,High,2,15,Male,High,NaN,1
2,182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,3,20,Female,High,Tertiary,2
3,210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,4,22,Female,Medium,Tertiary,1
4,199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,5,22,Female,Medium,Tertiary,1


Remove the `student_id` column.

In [9]:
df = df.drop(columns=['student_id'])

KeyError: "['student_id'] not found in axis"

In [11]:
df.head().T

,0,1,2,3,4
jamb_score,192,207,182,210,199
study_hours_per_week,22,14,29,29,12
attendance_rate,78,88,87,99,98
teacher_quality,4,4,2,2,3
distance_to_school,12.4,2.7,9.6,2.6,8.8
school_type,Public,Public,Public,Public,Public
school_location,Urban,Rural,Rural,Urban,Urban
extra_tutorials,Yes,No,Yes,No,No
access_to_learning_materials,Yes,Yes,Yes,Yes,Yes
parent_involvement,High,High,High,Medium,Medium


Filling missing values with zeros.

In [13]:
 print(df.isnull().sum())

jamb_score                        0
study_hours_per_week              0
attendance_rate                   0
teacher_quality                   0
distance_to_school                0
school_type                       0
school_location                   0
extra_tutorials                   0
access_to_learning_materials      0
parent_involvement                0
it_knowledge                      0
age                               0
gender                            0
socioeconomic_status              0
parent_education_level          891
assignments_completed             0
dtype: int64


In [14]:
df.parent_education_level = df.parent_education_level.fillna(0)

In [15]:
df.isnull().sum()

,0
jamb_score,0
study_hours_per_week,0
attendance_rate,0
teacher_quality,0
distance_to_school,0
school_type,0
school_location,0
extra_tutorials,0
access_to_learning_materials,0
parent_involvement,0


Do train/validation/test split with 60%/20%/20% distribution.

Use the train_test_split function and set the random_state parameter to 1.

Use DictVectorizer(sparse=True) to turn the dataframes into matrices.

In [17]:
df['jamb_score'] = np.log1p(df['jamb_score'])

In [18]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [19]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [21]:
y_train = df_train.jamb_score.values
y_val = df_val.jamb_score.values
y_test = df_test.jamb_score.values

In [22]:
del df_train['jamb_score']
del df_val['jamb_score']
del df_test['jamb_score']

In [24]:
df_train.T

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
study_hours_per_week,20,11,31,29,28,31,32,34,21,0,...,4,5,18,7,36,1,3,17,25,18
attendance_rate,72,80,82,79,96,78,78,100,79,70,...,83,88,79,95,88,79,87,96,74,67
teacher_quality,3,2,1,1,2,3,3,2,2,1,...,3,3,1,2,2,3,1,4,2,2
distance_to_school,4.4,3.3,8.3,15.8,8.9,9.3,8.0,15.4,12.9,5.8,...,14.7,5.3,4.7,9.1,7.2,12.2,9.6,13.7,3.4,10.5
school_type,Public,Public,Public,Public,Private,Public,Public,Public,Public,Public,...,Public,Public,Public,Public,Private,Public,Public,Private,Public,Public
school_location,Urban,Urban,Urban,Rural,Rural,Urban,Urban,Rural,Urban,Rural,...,Urban,Rural,Rural,Rural,Urban,Urban,Urban,Urban,Rural,Urban
extra_tutorials,No,Yes,Yes,Yes,Yes,No,Yes,No,No,Yes,...,No,Yes,No,No,No,No,No,No,No,Yes
access_to_learning_materials,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,...,Yes,No,Yes,Yes,No,No,No,Yes,No,Yes
parent_involvement,Medium,Medium,Low,Low,Medium,Medium,Low,Medium,Medium,Low,...,Medium,Medium,Medium,Low,High,Low,Medium,High,High,Low
it_knowledge,Low,High,High,Low,Low,Medium,Low,Medium,High,High,...,High,Low,Low,Low,Medium,High,Medium,Medium,High,Medium


In [25]:
# Turn the dataframes into matrices

train_dicts = df_train.to_dict(orient='records')
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(train_dicts)

In [26]:
X_train

<3000x29 sparse matrix of type '<class 'numpy.float64'>'
	with 45000 stored elements in Compressed Sparse Row format>

In [27]:
val_dicts = df_val.to_dict(orient='records')
X_val = dv.transform(val_dicts)

#Question 1

Let's train a decision tree regressor to predict the jamb_score variable.

  * Train a model with max_depth=1.

Which feature is used for splitting the data?

  * study_hours_per_week
  * attendance_rate
  * teacher_quality
  * distance_to_school


In [28]:
model = DecisionTreeRegressor(max_depth=1)
model.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=1)

In [29]:
print(export_text(model, feature_names=list(dv.get_feature_names_out())))

|--- study_hours_per_week <= 18.50
|   |--- value: [5.02]
|--- study_hours_per_week >  18.50
|   |--- value: [5.21]



`study_hours_per_week` is used for splitting the data

#Question 2

Train a random forest regressor with these parameters:

* n_estimators=10
* random_state=1
* n_jobs=-1 (optional - to make training faster)

What's the RMSE of this model on the validation data?

* 22.13
* 42.13
* 62.13
* 82.12


In [37]:
rf_model = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)

In [38]:
rf_model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=1)

In [39]:
y_pred = rf_model.predict(X_val)

In [44]:
rmse = mean_squared_error(y_val, y_pred, squared=False).round(2)

print("The RMSE of the Random Forest model on validation is:", rmse)

The RMSE of the Random Forest model on validation is: 0.24


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


#Question 3

Now let's experiment with the `n_estimators` parameter

* Try different values of this parameter from 10 to 200 with step 10.
* Set `random_state` to `1`.
* Evaluate the model on the validation dataset.

After which value of `n_estimators` does RMSE stop improving? Consider 3 decimal places for calculating the answer.

* 10
* 25
* 80
* 200


In [45]:
n_estimators_values = list(range(10, 201, 10))

In [46]:
best_rmse = float('inf')
best_n_estimators = None

for n_estimators in n_estimators_values:

    rf_model = RandomForestRegressor(n_estimators=n_estimators, random_state=1, n_jobs=-1)

    rf_model.fit(X_train, y_train)

    y_pred = rf_model.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)

    if rmse < best_rmse:
        best_rmse = rmse
        best_n_estimators = n_estimators

print("The best n_estimators value with the lowest RMSE is:", best_n_estimators)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

The best n_estimators value with the lowest RMSE is: 200


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


#Question 4

Let's select the best `max_depth`:

* Try different values of `max_depth`: `[10, 15, 20, 25]`
* For each of these values,
  * try different values of `n_estimators` from 10 till 200 (with step 10)
  * calculate the mean RMSE
* Fix the random seed: `random_state=1`

What's the best `max_depth`, using the mean RMSE?

* 10
* 15
* 20
* 25


In [47]:
max_depth_values = [10, 15, 20, 25]

In [55]:
best_mean_rmse = float('inf')
best_max_depth = None

for max_depth in max_depth_values:
    rmse_values = []

    for n_estimators in n_estimators_values:
        rf_model = RandomForestRegressor(max_depth=max_depth, n_estimators=n_estimators, random_state=1, n_jobs=-1)
        rf_model.fit(X_train, y_train)
        y_pred = rf_model.predict(X_val)

        rmse = root_mean_squared_error(y_val, y_pred)
        rmse_values.append(rmse)

    mean_rmse = np.mean(rmse_values)

    if mean_rmse < best_mean_rmse:
        best_mean_rmse = mean_rmse
        best_max_depth = max_depth

print("The best max_depth based on mean RMSE is:", best_max_depth)

KeyboardInterrupt: 

#Question 5

We can extract feature importance information from tree-based models.

At each step of the decision tree learning algorithm, it finds the best split. When doing it, we can calculate "gain" - the reduction in impurity before and after the split. This gain is quite useful in understanding what are the important features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the feature_importances_ field.

For this homework question, we'll find the most important feature:

* Train the model with these parameters:
  * n_estimators=10,
  * max_depth=20,
  * random_state=1,
  * n_jobs=-1 (optional)
* Get the feature importance information from this model

What's the most important feature (among these 4)?

* study_hours_per_week
* attendance_rate
* distance_to_school
* teacher_quality


In [51]:
rf_model = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=20, n_estimators=10, n_jobs=-1, random_state=1)

In [52]:
feature_importances = rf_model.feature_importances

In [53]:
feature_importance_dict = dict(zip(dv.feature_names_, feature_importances))

In [54]:
most_important_feature = max(feature_importance_dict, key=feature_importance_dict.get)

print("The most important feature is:", most_important_feature)

The most important feature is: study_hours_per_week


#Question 6

Now let's train an XGBoost model! For this question, we'll tune the eta parameter:

* Install XGBoost
* Create DMatrix for train and validation
* Create a watchlist
* Train a model with these parameters for 100 rounds:

xgb_params = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

Now change `eta` from `0.3` to `0.1`.

Which eta leads to the best RMSE score on the validation dataset?

* 0.3
* 0.1
* Both give equal value


In [71]:
features = dv.feature_names_
features = [f.replace("<", "less") for f in features]

dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)

In [72]:
watchlist = [(dtrain, 'train'), (dval, 'val')]

In [73]:
xgb_params = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,

    'objective': 'reg:squarederror',
    'nthread': 8,

    'seed': 1,
    'verbosity': 1,
}

In [74]:
model = xgb.train(xgb_params, dtrain, num_boost_round=100,
                  verbose_eval=5,
                  evals=watchlist)

[0]	train-rmse:0.24669	val-rmse:0.25512
[5]	train-rmse:0.20045	val-rmse:0.23358
[10]	train-rmse:0.18206	val-rmse:0.23357
[15]	train-rmse:0.17105	val-rmse:0.23465
[20]	train-rmse:0.16134	val-rmse:0.23628
[25]	train-rmse:0.14975	val-rmse:0.23772
[30]	train-rmse:0.14221	val-rmse:0.23892
[35]	train-rmse:0.13515	val-rmse:0.23999
[40]	train-rmse:0.12792	val-rmse:0.24152
[45]	train-rmse:0.12104	val-rmse:0.24274
[50]	train-rmse:0.11649	val-rmse:0.24399
[55]	train-rmse:0.10980	val-rmse:0.24563
[60]	train-rmse:0.10486	val-rmse:0.24727
[65]	train-rmse:0.09920	val-rmse:0.24844
[70]	train-rmse:0.09386	val-rmse:0.24961
[75]	train-rmse:0.08946	val-rmse:0.25048
[80]	train-rmse:0.08409	val-rmse:0.25105
[85]	train-rmse:0.07931	val-rmse:0.25161
[90]	train-rmse:0.07467	val-rmse:0.25253
[95]	train-rmse:0.07175	val-rmse:0.25276
[99]	train-rmse:0.06899	val-rmse:0.25306


In [75]:
y_pred = model.predict(dval)

In [76]:
y_pred

array([5.3364153, 4.9522047, 5.078388 , 4.723369 , 5.504252 , 5.4063945,
       5.1118484, 5.5457096, 5.131741 , 5.1590447, 5.2843175, 5.0607705,
       5.3251367, 5.3052273, 5.074646 , 4.9381356, 5.026344 , 4.9436326,
       5.223596 , 4.929638 , 5.1201434, 5.1288815, 5.0999427, 5.372225 ,
       5.385012 , 5.3102264, 4.998733 , 5.2648973, 4.8384194, 5.08374  ,
       5.1090717, 5.3850374, 4.926511 , 5.1447673, 5.3393474, 4.963443 ,
       5.0406837, 5.1223755, 5.007368 , 5.143409 , 5.1276846, 4.9429965,
       5.176834 , 5.125158 , 4.898773 , 4.8970294, 5.2155366, 5.2681413,
       4.945432 , 5.0391774, 4.9269753, 5.081991 , 4.9409604, 5.47122  ,
       5.5797997, 5.596744 , 4.980116 , 5.0206556, 5.059091 , 4.985795 ,
       5.2813516, 5.0930758, 4.9387827, 5.217643 , 4.9856687, 5.3148174,
       5.664032 , 5.292664 , 5.322469 , 4.8007   , 5.2775893, 4.779554 ,
       5.094006 , 5.1351895, 5.273836 , 4.772467 , 5.402636 , 5.089718 ,
       4.887382 , 5.075885 , 5.213573 , 5.1560054, 

In [77]:
rmse_eta_0_3 = root_mean_squared_error(y_val, y_pred)
print("RMSE with eta=0.3:", rmse_eta_0_3)

RMSE with eta=0.3: 0.25305666864655285


In [78]:
# for eta=0.1

xgb_params = {
    'eta': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

In [79]:
model = xgb.train(xgb_params, dtrain, num_boost_round=100,
                  verbose_eval=5,
                  evals=watchlist)

[0]	train-rmse:0.26239	val-rmse:0.26670
[5]	train-rmse:0.23195	val-rmse:0.24586
[10]	train-rmse:0.21489	val-rmse:0.23803
[15]	train-rmse:0.20393	val-rmse:0.23386
[20]	train-rmse:0.19596	val-rmse:0.23250
[25]	train-rmse:0.18999	val-rmse:0.23132
[30]	train-rmse:0.18474	val-rmse:0.23093
[35]	train-rmse:0.18060	val-rmse:0.23150
[40]	train-rmse:0.17579	val-rmse:0.23195
[45]	train-rmse:0.17257	val-rmse:0.23235
[50]	train-rmse:0.16890	val-rmse:0.23249
[55]	train-rmse:0.16483	val-rmse:0.23230
[60]	train-rmse:0.16104	val-rmse:0.23259
[65]	train-rmse:0.15730	val-rmse:0.23312
[70]	train-rmse:0.15455	val-rmse:0.23362
[75]	train-rmse:0.15214	val-rmse:0.23406
[80]	train-rmse:0.15067	val-rmse:0.23445
[85]	train-rmse:0.14860	val-rmse:0.23492
[90]	train-rmse:0.14564	val-rmse:0.23526
[95]	train-rmse:0.14379	val-rmse:0.23550
[99]	train-rmse:0.14238	val-rmse:0.23578


In [83]:
y_pred2 = model.predict(dval)

In [84]:
rmse_eta_0_1 = root_mean_squared_error(y_val, y_pred2)
print("RMSE with eta=0.1:", rmse_eta_0_1)

RMSE with eta=0.1: 0.23577957697139892


In [85]:
if rmse_eta_0_3 < rmse_eta_0_1:
    best_eta = 0.3
elif rmse_eta_0_1 < rmse_eta_0_3:
    best_eta = 0.1
else:
    best_eta = "Both give equal value"

print("The best eta is:", best_eta)

The best eta is: 0.1
